In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import os, re
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import nltk
from gensim.models.doc2vec import TaggedDocument
import gensim.models.doc2vec
import pickle
from gensim.models import Doc2Vec

C:\Users\Hesham\AppData\Local\Programs\Python\Python36\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [34]:
data = pd.read_csv('./data_clean.csv')
with open('./docs_trigram.pkl', 'rb') as file:
    docs = pickle.load(file)
len(docs)

37092

In [35]:
with open('./d2v_model_trained.pkl', 'rb') as file:
    model = pickle.load(file)
    print(model.vector_size, model.window, model.min_count)

200 9 20


C:\Users\Hesham\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `min_count` (Attribute will be removed in 4.0.0, use self.vocabulary.min_count instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [18]:
strictness = 0.7
limit = 10
minimum = 3
max_depth = 3

# def get_recommended(url):
#     results = dict()
#     depth = 0
#     ms = model.docvecs.most_similar(url, topn=limit)
#     key = [url]

#     filtered = [r for r in ms if r[1] > strictness]
#     if(len(filtered) < minimum):
#         filtered = ms[:minimum]
#     return filtered

def get_recommended(url):
    results = dict()
    depth = 0
    keys = [url]
    n=1
    
    results = dict()
    while True:
        m = model.docvecs.most_similar(keys, topn=n)[-1]
        if '.' in m[0]:
            if not m[0] in results.keys():
                results[m[0]] = m[1]
            else:
                results[m[0]] = 2 / (1/results[m[0]] + 1/m[1])
            if len(results.items()) >= limit or (m[1] < strictness and not len(results) < minimum):
                break
#         elif depth < max_depth and m[1] > strictness:
#             keys.append(m[0])
#             depth += 1
#             n=0
        n += 1
    
    results = sorted(results.items(), key=lambda x:x[1], reverse=True)
    
    filtered = [m for m in results if m[1] > strictness and '.' in m[0]]
    
    if(len(filtered) < minimum):
        filtered = results[:minimum]
    elif len(filtered) > limit:
        filtered = results[:limit]
    return filtered

def get_text(url):
    return '"'+str(data.text[data.url == url].values[0].encode("utf-8")).replace('"', '""')+'"'

def get_label(url):
     return '"'+str(data.label[data.url == url].values[0].encode("utf-8")).replace('"', '""')+'"'

In [39]:
rand = np.arange(len(docs))
np.random.shuffle(rand)
rand = rand[:10]

with open('./output.csv', 'w') as file:
    file.write('match,url,label,text\n')
    for doc in np.array(docs)[rand]:
        url = doc[1][0]
        label = get_label(url)
        ms = get_recommended(url)
        
        file.write(','+url+','+label+','+get_text(url)+'\n')
#         if kw:
#             k = '-'.join(kw)
#             file.write(',,'+k+'\n')
        for m in ms:
            file.write('%.2f%%,%s,%s,%s\n'%(m[1]*100,m[0],get_label(m[0]),get_text(m[0])))
        file.write(',,\n')

In [31]:
n = 0
with open('./output.json', 'w') as file:
    file.write('{\n')
    for doc in np.array(docs):
        url = doc[1][0]
        ms = get_recommended(url)
        file.write('"'+url+'":[')
        file.write(','.join(['["%s",%.2f]'%(m[0],m[1]) for m in ms]))
        file.write('],\n')
        n += 1
        if n % 1000 == 0:
            print(n)
    file.write('}')

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000


In [26]:
df = pd.read_csv('output.csv')
urls = df[df.match.isnull()].url[~df.url.isnull()].unique()
len(urls)

400

In [80]:
with open('./output.csv', 'w') as file:
    file.write('match,url,label,text\n')
    for u in urls:
        print(u)
        for doc in np.array(docs):
            url = doc[1][0]
            if url == u:
                ms = get_recommended(url)
                file.write(','+url+','+str(data[data.url==url].label.values[0].encode("utf-8"))+','+get_text(url)+'\n')
        #         if kw:
        #             k = '-'.join(kw)
        #             file.write(',,'+k+'\n')
                for m in ms:
#                     print(m)
                    file.write('%.2f%%,%s,%s,%s\n'%(m[1]*100,m[0],str(data[data.url==m[0]].label.values[0].encode("utf-8")),get_text(m[0])))
                file.write(',,\n')

haugenhandgunleather.com
eaglescreens.com.au
millerscheesehouse.com
magicmicusa.com
malespecies.com
donnabsbakery.com
ragen-mendenhall.pixels.com
cowdogsaddles.com
iserv.net/~gtractor/
pitbullmotorcyclelifts.com
vtkeepsake.com
pacificshaving.com
eatmyflames.com
carolinacookie.com
napaonline.com
southerntraditionalfoods.com
dahliabarn.com
grassrootsmeats.com
goodsports.com
repeaterstore.com
merimask.com
accessorygeeks.com
equalizerhitch.com
shirtmagic.com
devittliptakdesign.com
discounthairsystems.com
schopper.ch
enchantingbeads.com
greekcity.com
bobpaulrarecoins.com
headwaymusicaudio.com
barcelonabathandbody.com
audubonart.com
atvspecialtiesinc.com
pastperfect.com
dollhouse-bedding-drapes.com
findgiftclubs.com
dogchristmasstockings.com
colonialwoodworks.net
recordsbymail.com
tropicalfruitshop.com
swisspanoramashop.com
seajunk.com
rugsdirect.co.uk
bmcglobal.com
esthersnc.com
sallymitchell.com
cookiegluck.com
ingridandisabel.com
lids.com
rayersinc.com
cbcshop.ca
esmodels.co.uk
growjourne